In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np
import pandas as pd

In [10]:
STORM_NUMBER = 6
TITLE = "22-23 January 2022"
SUBTITLE = "8 AM 23 January 2022"
sts = utc(2022, 1, 23, 6)
ets = utc(2022, 1, 30, 20, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

/opt/miniconda3/envs/prod/lib/python3.8/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/opt/miniconda3/envs/prod/lib/python3.8/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [13]:
cull = ['IA-AL-3', 238, 319]
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [

]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=2)
res = mp.postprocess(filename='220123.png')
mp.close()

     state  wfo   val      lon      lat                             geo  \
0       NY  BUF  16.0 -75.6200  43.9700  POINT (1914645.136 176616.083)   
1       NY  BUF  14.0 -76.1800  43.8500  POINT (1876129.664 150521.630)   
2       NY  BUF  13.0 -79.0300  42.6500  POINT (1691980.871 -42152.463)   
5       NY  BUF  10.6 -78.9600  42.6900  POINT (1696357.608 -36349.618)   
13      MI  APX   7.1 -85.0500  44.9000   POINT (1169168.635 97306.807)   
...    ...  ...   ...      ...      ...                             ...   
1754    MO  PAH   0.0 -90.4100  36.7600  POINT (854233.660 -866613.971)   
1755    MO  SGF   0.0 -94.0264  38.2000  POINT (522361.809 -736981.865)   
1756    MO  SGF   0.0 -93.5853  37.3131  POINT (567891.952 -832356.818)   
1757    MO  SGF   0.0 -93.1969  36.5258  POINT (608738.041 -916792.491)   
1758    MO  SGF   0.0 -93.4167  37.6000  POINT (580451.159 -799428.038)   

      used_for_analysis  nwsli  plotme source  xcell  ycell  
0                  True      0    Tru